In [ ]:
import math
import numpy as np

def parseRDD(point):
    """ Converte um ponto de metricas da base de dados para uma tupla (id, vetor de floats).
        Recebe um ponto onde o primeiro campo eh o id do vertice os 13 seguintes sao os valores para cada metrica
        e retorna uma tupla composta pelo id e pelas 13 metricas (vetor de floats).
    Args:
        point (str): uma string onde os termos estao separados por ',', sendo o primeiro campo o id do vertice e 
        os 13 seguintes os valores para cada metrica
    Returns:
        (id, []): uma tupla composta pelo id do vertice e uma lista dos valores de 13 metricas
    """
    data = point.split(';')
    floatMetrics = [float(i) for i in data[1:]]
    return (data[0], floatMetrics)

def isNotZero(parsedPoint):
    """ Retorna true se o ponto contem alguma metrica diferente de 0.
    Args:
        parsedPoint (str, []): uma tupla composta pelo id do vertice e lista de metricas
    Returns:
        bool: True se a lista contém pelo menos um valor nao nulo ou False se a somatoria da lista eh nula
    """
    return sum(parsedPoint[1]) > 0

def normalize(parsedPoint, means, standardDeviations, maxs, mins, normalizationType):
    """ Normaliza um ponto. Recebe um ponto cujas valores maximo e minimo para as metricas 
        podem ser muito amplos e retorna um ponto cujas metricas estao entre 0 e 1.0.
    Args:
        parsedPoint (str, []): uma tupla composta pelo id do vertice e lista de metricas
        means ([]): lista de medias das metricas
        standardDeviations (list): lista de desvios-padrao das metricas
        maxs ([]): lista de maximos das metricas
        mins ([]): lista de minimos das metricas
        normalizationType (str): tipo de normalizacao ('reescaling' | 'standard_score')
    Returns:
        (str, []): uma tupla de id (str) e metricas normalizadas ([])
    """
    nodeId = parsedPoint[0]
    metrics = parsedPoint[1]
    numberOfMetrics = len(means) # numero de metricas == numero de medias == numero de desvios-padrao
    if normalizationType == 'standard_score':
        normalizedMetrics = [(metrics[i] - means[i])/standardDeviations[i] for i in range(numberOfMetrics)]
    elif normalizationType == 'reescaling':
         normalizedMetrics = [(metrics[i] - mins[i])/(maxs[i]-mins[i]) for i in range(numberOfMetrics)]
    return (nodeId, normalizedMetrics)

def euclidianDistance(pointA, pointB):
    """ Calcula a distancia euclidiana entre dois pontos. Recebe dois pontos e retorna a distancia
        euclidiana entre eles.
    Args:
        pointA ([]): lista de floats
        pointB ([]): lista de floats
    Returns:
        float: a distancia entre dois pontos
    """
    numberOfMetrics = len(pointA) # numero de metricas de A e B eh igual a 13
    squaredDifferenceBetweenMetrics = [math.pow(pointA[i] - pointB[i], 2) for i in range(numberOfMetrics)]
    return math.sqrt(sum(squaredDifferenceBetweenMetrics))

def chooseRandomPoints(points, numberOfPoints):
    """ Escolhe um ponto aleatorio dentre uma lista de pontos. 
        Recebe uma lista de pontos e retorna um ponto dessa lista.
    Args:
        points (RDD): RDD de pontos
        numberOfPoints (int): numero de pontos a serem escolhidos
    Returns:
        points ([[]]): uma lista de lista de metricas representando n pontos
    """
    return points.takeSample(False, numberOfPoints)
    
def areCentroidsDifferent(pointA, pointB):
    """ Verifica se dois pontos sao diferentes, com base nas listas de metricas.
        Recebe dois pontos e returna True ou False.
    Args:
        pointA: um ponto do tipo [], em que contem metricas do tipo float
        ponttB: um ponto do tipo [], em que contem metricas do tipo float
    Returns:
        bool: True se pointA[i] != pointB[i] para todo i ou False caso contrario
    """
    for i in range(len(pointA)):
        if pointA[i] != pointB[i]:
            return True
    return False
        
def kmeans(data, k, iteractions):
    """ Base do bisect-kmeans. Recebe uma RDD de dados, um k=2 e o numero de iteracoes maximo e retorna dois clusters.
    Args:
        data (RDD): RDD de pontos do tipo (str, [])
        k (int): numero de clusters a serem gerados. Padrao para bisect é 2 por vez
        iteractions (int): numero de interacoes maximo
    Returns:
        clusters (RDD): um RDD contendo dois clusters. 
                        Cada ponto eh mapeado para (idClusters, idPonto, metricas)
                        idClusters eh ou 0 ou 1
                        metricas eh uma lista de floats
    """

    # seleciona k pontos aleatorios e atribui a lista inicial de centroides
    centroids = [p[1] for p in chooseRandomPoints(data, k)] 
    
    # roda kmeans ateh o limite de iteractions
    for i in range(iteractions):
        print('\tkmeans interaction: %s' %(i + 1))
        # calcula distancia entre cada ponto (segundo elemento da tupla) e os centroides, e atribui a cada ponto o id do centroide cuja distanca eh menor
        clustersRDD = data.map(lambda x:(np.argmin([euclidianDistance(x[1], c) for c in centroids]), x[0], x[1]))
        
        # refaz lista de centroides, calculando o ponto medio dos pontos de cada cluster
        newCentroids = (clustersRDD
                        .map(lambda x:(x[0], [x[2],1])) # mapeia cada ponto para (indice do centroide, [vetor de metricas, 1])
                        .reduceByKey(lambda x,y:([(np.array(x[0]) + np.array(y[0])),(x[1] + y[1])])) # reduz conjunto de pontos com mesmo indice do centroide para a somatoria dos vetores de metricas
                        .map(lambda x:list(np.array(x[1][0])/(x[1][1]))) # divide a somatoria dos vetores pelo numero de pontos
                       ).collect()
        
        # caso numero de centroides mude, seleciona centroides faltantes de forma aleatoria
        # isso pode ocorrer se o reduceByKey da linha 108 nao tiver a chave de um dos centroids
        if len(newCentroids) != len(centroids):
            print('\t\tnumber of centroids has changed (fixing it now)')
            diff = len(centroids) - len(newCentroids)
            newCentroids.extend = [p[1] for p in chooseRandomPoints(data, diff)]
                
        # verifica se centroides mudaram
        centroidsHaveChanged = [areCentroidsDifferent(centroids[i], newCentroids[i]) for i in range(len(centroids))]
        
        if True not in centroidsHaveChanged:
            print('\t\tcentroids have not changed')
            return clustersRDD
        else:
            centroids = newCentroids
    return clustersRDD

def getClustersOrderedBySize(clusters):
    """ Verifica qual dos clusters possui mais elementos e os retorna ordenado pela quantidade crescente de elementos.
    Args:
        clusters (RDD): RDD contendo pontos cujas chaves ou sao 0 ou sao 1
    Returns:
        clusterMenor (RDD), clusterMaior (RDD): Dois RDDs de pontos em que o primeiro eh o menor e o segundo o maior
    """
    cluster1 = clusters.filter(lambda x:x[0] == 0)
    cluster2 = clusters.filter(lambda x:x[0] == 1)
    if cluster1.count() > cluster2.count():
        return cluster2, cluster1
    else:
        return cluster1, cluster2
    
def bisectKmeans(data, k, iteractions, forceAllClustersWithPoints):
    """ Algoritmo principal do bisect-kmeans. Recebe uma RDD de dados, o numero de clusters a serem gerados,
        o numero de iteracoes maximo do subalgoritmo kmeans e um boleano indicando se pode haver clusters vazios
    Args:
        data (RDD): RDD de pontos do tipo (idPonto, metricas), onde idPonto eh um str e metricas eh uma lista de floats
        k (int): numero de clusters
        iteractions (int): numero de iteracoes do subalgoritmo k-means
        forceAllClustersWithPoints (bool): um boleano que indica que nao sera permitido clusters vazios
    Returns:
        finalClusters ([]): lista de pontos do tipo (idPonto, metricas), 
                            len(finalClusters) eh garantido ser k, se forceAllClusterWithPoints for True
                            cada cluster contem x pontos
    """
    # inicializa lista final de clusters
    finalClusters = []
    
    # todos os pontos a serem divididos
    clusterToSplit = data
    i = 0
    
    while(i != k - 1):
        print('bisect interaction: %s - clusters: %s' %(i + 1, len(finalClusters)))
        # roda kmeans para k = 2
        dualClustersRDD = kmeans(data=clusterToSplit, k=2, iteractions=iteractions)

        # melhor cluster eh o menor e o pior cluster eh o maior
        bestClusterRDD, worstClusterRDD = getClustersOrderedBySize(dualClustersRDD)
        
        print('size(best cluster) %s' %bestClusterRDD.count())
        print('size(worst cluster) %s' %worstClusterRDD.count())
        
        if forceAllClustersWithPoints and bestClusterRDD.count() == 0:
            print('found empty cluster - repeating proccess')
        else:
            # insere melhor cluster na lista final de clusters
            finalClusters.append(bestClusterRDD)

            # incrementa numero de clusters
            i += 1

            # insere o pior cluster para rodar novamente no kmeans
            clusterToSplit = worstClusterRDD.map(lambda x:(x[1],x[2]))
        
    finalClusters.append(worstClusterRDD)        
    return finalClusters

In [ ]:
fileName = ('metricas_t.csv')
rawRDD = sc.textFile(fileName)
metricsHeader = rawRDD.take(1)[0]
metricsRDD = (rawRDD
              .filter(lambda x: x != metricsHeader)
              .map(lambda x:parseRDD(x))
              .filter(lambda x: isNotZero(x))
             )

In [ ]:
means = []
maxs = []
mins = []
stdevs = []
for i in range(13):
    metricI = metricsRDD.map(lambda x: x[1][i])
    maxs.append(metricI.max())
    mins.append(metricI.min())
    means.append(metricI.mean())
    stdevs.append(metricI.stdev())

In [ ]:
normalizedMetricsRDD = metricsRDD.map(lambda x:normalize(x, means, stdevs, maxs, mins, 'standard_score'))

In [ ]:
clusters = bisectKmeans(data=normalizedMetricsRDD, iteractions=15, k=10, forceAllClustersWithPoints=True)

In [ ]:
tamanhos = []
for index, c in enumerate(clusters):
    tamanhos.append(c.count())
    print('cluster %s possui %s pontos' %(index + 1, tamanhos[index]))
     c.saveAsTextFile('cluster/%s' %index)
    
print('------------------------\ntotal possui %s pontos' %sum(tamanhos))